In [1]:
from data import UnraveledDataset
from data import load_data
from models import MLP
from train import *
import torch
from torchvision import datasets
import utils_graphs as ug
from utils_tda import *

## Model Instantiation

For this experiment a flatten MNIST is used and 2 hidden layers MLP is trained

In [2]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
device

'cuda'

In [4]:
dataclass = UnraveledDataset
dataset = datasets.MNIST
batch_size = 1500
dataloader_train, dataloader_val = load_data(dataclass, dataset, download = False)

In [5]:
kwargs = dict(
    n_features=28 * 28,
    hidden_layer_sizes=(100, 100),
    n_targets=10
)
loss_inst = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam

In [6]:
model = MLP(**kwargs)
model.to(device)

MLP(
  (module_list): ModuleList(
    (0): Linear(in_features=784, out_features=100, bias=True)
    (1): Linear(in_features=100, out_features=100, bias=True)
    (2): Linear(in_features=100, out_features=10, bias=True)
  )
)

In [7]:
train(model, dataloader_train, dataloader_val, loss = loss_inst, optimizer = optimizer, device = device)

Train accuracy this epoch = 0.7286333441734314
Validation accuracy this epoch = 0.8603999614715576
Validation loss this epoch = 0.545211672782898
Train accuracy this epoch = 0.8732333183288574
Validation accuracy this epoch = 0.8992999792098999
Validation loss this epoch = 0.36273708939552307
Train accuracy this epoch = 0.8999500274658203
Validation accuracy this epoch = 0.9106999635696411
Validation loss this epoch = 0.30610352754592896
Train accuracy this epoch = 0.9132833480834961
Validation accuracy this epoch = 0.918999969959259
Validation loss this epoch = 0.27519363164901733
Train accuracy this epoch = 0.9208166599273682
Validation accuracy this epoch = 0.9246999621391296
Validation loss this epoch = 0.25088831782341003
Train accuracy this epoch = 0.9290000200271606
Validation accuracy this epoch = 0.9311999678611755
Validation loss this epoch = 0.23294757306575775
Train accuracy this epoch = 0.9345333576202393
Validation accuracy this epoch = 0.9371999502182007
Validation loss 

In [8]:
model.to('cpu')

MLP(
  (module_list): ModuleList(
    (0): Linear(in_features=784, out_features=100, bias=True)
    (1): Linear(in_features=100, out_features=100, bias=True)
    (2): Linear(in_features=100, out_features=10, bias=True)
  )
)

## Persistence diagrams calculation

 A hundred data points per class are used to construct each class persistence diagram from which the distance of the persistence diagram of new data points will be calculated

In [9]:
batch = next(iter(dataloader_train))

In [10]:
n_obs_per_class = 100

labels = np.unique(batch[1])
# Take n_obs_per_class observation from each class in x_train:
xs = [batch[0][np.where(batch[1] == label)][:n_obs_per_class] for label in labels]

# Now we get the corresponding barycenters.


In [11]:
%%time
# Now we get the corresponding barycenters.
all_barycenters = [barycenters_of_set_from_deep_model(model, x, layers_id= None) for x in xs]

CPU times: total: 3min 33s
Wall time: 3min 17s


In [12]:
# Now we take 10 observations from the validation dataset
n_new_obs = 10
val_batch = next(iter(dataloader_val))
new_x = val_batch[0][:n_new_obs]

In [13]:
# We calculate topological uncertainty for each of the observations
res = topological_uncertainty(model, new_x, all_barycenters, layers_id= None)
res

array([0.01052685, 0.01028769, 0.00568619, 0.00871088, 0.00565445,
       0.00889411, 0.02230883, 0.00480581, 0.00474241, 0.03066175])

## Sensibility analysis
First we add gaussian noise to previous observation and see how topological uncertainty behaves under the increasing noise.

Then we use Fashion MNIST as new observations to ensure the uncertainty increases

In [14]:
for i in range(100):
    new_new_x = new_x + torch.tensor(np.array([np.random.normal(scale = 00.1*i, size = 784) for j in range(10)])).float()
    res = topological_uncertainty(model, new_new_x, all_barycenters, layers_id= None)
    print(res)

[0.01052685 0.01028769 0.00568619 0.00871088 0.00565445 0.00889411
 0.02230883 0.00480581 0.00474241 0.03066175]
[0.01081969 0.01064705 0.00757768 0.00667485 0.0059485  0.00795631
 0.02349286 0.00639459 0.00448364 0.02815942]
[0.00819986 0.00931028 0.00460874 0.00547386 0.0043531  0.01071629
 0.02616318 0.00325857 0.00505081 0.03500767]
[0.01154461 0.02432016 0.00703408 0.00398967 0.00359572 0.00776361
 0.02600962 0.00602203 0.00651329 0.03040852]
[0.00469005 0.01756099 0.00701978 0.0045655  0.00577613 0.00796533
 0.01444144 0.00241808 0.00704341 0.03099638]
[0.00472774 0.03447553 0.0112701  0.00401605 0.00828283 0.00753754
 0.02045229 0.01007731 0.01158967 0.02583326]
[0.00595807 0.01647268 0.01059332 0.00545032 0.01647268 0.00633246
 0.02345635 0.00760902 0.01231128 0.03719979]
[0.01128017 0.02074992 0.01535739 0.01564726 0.00478089 0.00879311
 0.02330974 0.00530172 0.008774   0.04507123]
[0.02097506 0.0423248  0.02328161 0.00662354 0.00795511 0.00736957
 0.05552054 0.02106393 0.0076

In [15]:
fdataclass = UnraveledDataset
fdataset = datasets.FashionMNIST
fbatch_size = 100
_, fmnist_x = load_data(fdataclass, fdataset, download = False, batch_size = fbatch_size)

In [16]:
fbatch_x = next(iter(fmnist_x))[0][:n_new_obs]

In [17]:
%%time
res2 = topological_uncertainty(model, fbatch_x, all_barycenters, layers_id= None)

CPU times: total: 2.94 s
Wall time: 1.93 s


In [18]:
res2

array([0.03077389, 0.00492971, 0.04925872, 0.02009449, 0.0463532 ,
       0.03694918, 0.05156962, 0.03711148, 0.07797239, 0.04575797])